In [10]:
import pandas as pd 
import sqlalchemy as sql

In [3]:
import geopandas

# LUDU QC

## Dounload the data

In [4]:
creds = {'un': 'SANDAGNET\\cra' , 'pw': '' }
engine = sql.create_engine('mssql+pymssql://%s:%s@sql2014b8/WS' % (creds['un'], creds['pw']))
lu21 =pd.read_sql('SELECT [OBJECTID],[LCKey],[subParcel], [parcelID], [apn8],[lu],[du],[genOwnID],[MGRA],[Region] FROM [WS].[gis].[LUDU2021]',
engine.connect())
lu21

,OBJECTID,LCKey,subParcel,parcelID,apn8,lu,du,genOwnID,MGRA,Region
0,1,799710,523391400,5233914,,4118.0,0,1.0,23668,0.0
1,2,129599,507660300,5076603,22553006,1110.0,1,500.0,20050,0.0
2,3,908523,532796600,5327966,67933019,1110.0,1,500.0,19833,0.0
3,4,645745,126833900,1268339,,4118.0,0,1.0,13944,0.0
4,5,688043,44718800,447188,38130418,1110.0,1,500.0,6284,0.0
...,...,...,...,...,...,...,...,...,...,...
837727,837728,47712,509080700,5090807,,4118.0,0,1.0,7611,0.0
837728,837729,896076,531979900,5319799,16947229,7606.0,0,500.0,19223,0.0
837729,837730,146619,58906100,589061,12752217,8001.0,0,500.0,17495,0.0
837730,837731,875113,528840100,5288401,,4118.0,0,1.0,17992,0.0


In [5]:
lu21[lu21['LCKey'] == 916868]

,OBJECTID,LCKey,subParcel,parcelID,apn8,lu,du,genOwnID,MGRA,Region
837054,837055,916868,72070201,720702,76004500,7207.0,113,28.0,10795,0.0


# LUDU Codes To Be Analyzed
Here I am going through SANDAG's land use codes and seeing which should not have any dwelling units on them and reviewing from there. \
SANDAG's landuse codes can be found here: https://www.sandag.org/resources/maps_and_gis/gis_downloads/downloads/codes/Land_Use_Definitions.html?msclkid=2b916e35b4f711ec9945f63a2dbf1b8f

## Non-residential Codes to Checks

In [6]:
non_res_lu_codes = {'group_quarters':[1400, 1409], 
'hotels': [1500, 1501, 1502, 1503],
'vacant':[9101],
'transportation':[4101, 4102, 4103, 4104, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4120],
'cemetery':[6101],
'parks_open_spaces':[7600, 7601, 7603, 7604, 7605, 7606, 7607, 7609],
'water':[9200,9201, 9202],
'Heavy Industry':[2000,2001],
'Light Industry':[2100,2101,2103,2104,2105],
'Extractice Industry':[2200,2201],
'Junkyard':[2300,2301],
'Airports':[4100,4101,4102,4103,4104],
'Commercial':[5000,5001,5002,5003,5004,5005,5006,5007,5008,5009],
'Office':[6000,6001,6002,6003],
'Public Services':[6100,6102,6103,6104,6105,6108,6109],
'Hospitals':[6500,6501,6502,6509],
'Military Use':[6700,6701,6702,6703],
'Schools':[6800,6801,6802,6803,6804,6805,6806,6807,6809],
'Commercial Recreation':[7200,7201,7202,7203,7204,7205,7206,7207,7208,7209,7210,7211],
'Parks':[7600,7601,7603,7604,7605,7606,7607,7609],
'Agriculture':[8000,8001,8002,8003],
'Vacant and Undeveloped Land':[9100,9101]
}

# Non-Residential Checks
This function will check the dweeling units in question and spit out the LCKey's that are incorrect 

In [7]:
flags = pd.DataFrame(columns=['Land Use', 'LCKey', 'du'])

In [8]:
for land_use_name in non_res_lu_codes.keys():
    lu21_list= lu21[lu21.lu.isin(non_res_lu_codes[land_use_name])]
    df = lu21_list[lu21_list['du'] > 0]
    df['Land Use'] = land_use_name
    final_df = df[['Land Use', 'LCKey', 'du']]
    flags = pd.concat([flags, final_df])

C:\Users\cra\AppData\Local\Temp\ipykernel_13756\2871836238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Land Use'] = land_use_name
C:\Users\cra\AppData\Local\Temp\ipykernel_13756\2871836238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Land Use'] = land_use_name
C:\Users\cra\AppData\Local\Temp\ipykernel_13756\2871836238.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveat

In [9]:
flags = flags.reset_index(drop=True)

In [10]:
flags.head(3)

,Land Use,LCKey,du
0,cemetery,488783,1
1,parks_open_spaces,855848,1
2,parks_open_spaces,889329,3


In [11]:
flags['SQL_LCKey'] = flags.apply(lambda row: f"'{row.LCKey}',", axis=1)

In [12]:
nonres_flags = flags[flags['du'] > 10].reset_index(drop=True)

In [13]:
# Sending to the GitHub Repo
nonres_flags.to_excel('C:/Users/cra/OneDrive - San Diego Association of Governments/QA_Repository/2022/2022-16 LUDU/flags.xlsx')

# Sending to Sharepoint
nonres_flags.to_excel("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-16 LUDU 2021/nonres_flags.xlsx", index=False)

# Residential Checks 
Here I will be doing an analysis on single family residential and seeing which codes have over five dwelling units.

## Codes to check

In [28]:
res_lu_codes = {'Single Family Residential':[1100, 1110, 1120, 1190] 
}

In [29]:
res_flags = pd.DataFrame(columns=['Land Use', 'lu', 'LCKey', 'du'])

In [30]:
for land_use_name in res_lu_codes.keys():
    lu21_list= lu21[lu21.lu.isin(res_lu_codes[land_use_name])]
    df = lu21_list[lu21_list['du'] > 0]
    df['Land Use'] = land_use_name
    final_df = df[['Land Use', 'lu', 'LCKey', 'du', 'genOwnID']]
    res_flags = pd.concat([res_flags, final_df])

C:\Users\cra\AppData\Local\Temp\ipykernel_13756\3691348485.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Land Use'] = land_use_name


In [31]:
res_flags['SQL_LCKey'] = res_flags.apply(lambda row: f"'{row.LCKey}',", axis=1)

In [32]:
res_flags = res_flags[res_flags['du'] > 5].reset_index(drop=True)

In [33]:
res_flags['lu'] = res_flags['lu'].astype('int')

In [37]:
res_flags = res_flags[res_flags['genOwnID'] != 41]

In [39]:
res_flags = res_flags[res_flags['du'] > 100]

In [43]:
res_flags.reset_index(drop=True)

,Land Use,lu,LCKey,du,genOwnID,SQL_LCKey
0,Single Family Residential,1120,709182,166,500.0,"'709182',"
1,Single Family Residential,1120,872303,108,500.0,"'872303',"
2,Single Family Residential,1120,919076,150,30.0,"'919076',"
3,Single Family Residential,1120,466465,210,30.0,"'466465',"
4,Single Family Residential,1120,843349,108,500.0,"'843349',"
5,Single Family Residential,1120,872284,270,500.0,"'872284',"
6,Single Family Residential,1120,775091,116,500.0,"'775091',"
7,Single Family Residential,1120,918535,160,500.0,"'918535',"
8,Single Family Residential,1120,474686,300,500.0,"'474686',"
9,Single Family Residential,1120,873726,152,500.0,"'873726',"


In [45]:
# Sending to the GitHub Repo
res_flags.to_excel('C:/Users/cra/OneDrive - San Diego Association of Governments/QA_Repository/2022/2022-16 LUDU/res_flags.xlsx')

# Sending to Sharepoint
res_flags.to_excel("C:/Users/cra/San Diego Association of Governments/SANDAG QA QC - Documents/Projects/2022/2022-16 LUDU 2021/residential_flags_subset.xlsx", index=False)